In [ ]:
//#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Application.BoSSSpad;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "Benchmark" : buildname;
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
dbname = String.IsNullOrEmpty(dbname)? "dbname" : dbname;
string table_name = String.Concat(buildname, "_", dbname);

In [ ]:
string WFlowName = table_name;
var wfmgr = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm;
wfmgr.Init(WFlowName); wfmgr.Sessions

In [ ]:
wfmgr.Sessions.Where(s=>!s.SuccessfulTermination).ForEach(s=>s.Delete(true));
wfmgr.Init(WFlowName); wfmgr.Sessions

In [ ]:
/*
CG projection               -> ProjectDGField                               BoSSS.Foundation.ConstrainedDGprojection.ConstrainedDGField_Global.ProjectDGField
dyn Loadbalancing           -> DoLoadbalancing                              BoSSS.Solution.Application`1.DoLoadbalancing
AMR                         -> DoMeshAdaption                               BoSSS.Solution.Application`1.DoMeshAdaption
compile Cut-cell quadrature -> Rule_Compilation_SayeGaussRuleFactory        Rule_Compilation_SayeGaussRuleFactory
compile standard quadrature -> Rule_Compilation_StandardQuadRuleFactory     Rule_Compilation_StandardQuadRuleFactory
matrix assembly             -> XdgMatrixAssembly                            BoSSS.Solution.XdgTimestepping.XdgBDFTimestepping.AssembleMatrixCallback
agg basis init              -> Aggregation_basis_init_time                  BoSSS.Solution.AdvancedSolvers.XdgAggregationBasis.Update
solver init                 -> Solver_Init_time                             Slv Init
solver run                  -> Solver_Run_time                              Slv Iter
I/O                         -> SaveToDatabase                               BoSSS.Solution.Application`1.SaveToDatabase
*/

In [ ]:
static var PreComputeProfiling = new Dictionary<System.Guid,MethodCallRecord[]>();
foreach(var sess in wfmgr.Sessions){
    MethodCallRecord[] mcrs = null;
    try{
        mcrs = sess.GetProfiling();
    } catch (Exception ex){
        Console.WriteLine(ex.Message);
    }
    PreComputeProfiling.Add(sess.ID, mcrs);
}

In [ ]:
static class AddCols {
    static public MethodCallRecord[] GetMCR(ISessionInfo SI){
        MethodCallRecord[] mcrs = null;
        PreComputeProfiling.TryGetValue(SI.ID,out mcrs);
        return mcrs;
    }
    static private double TimeOnProc(MethodCallRecord mcr, string method){
        double time = 0;
        foreach(var child in mcr.FindChildren(method)){
            time += child.TimeSpentInMethod.TotalSeconds;
            //time += child.TimeSpentInMPIBlocking.TotalSeconds;
        }
        /*
        var child = mcr.FindChildren(method).Pick(0);
        time = child.TimeExclusive.TotalSeconds;
        */
        /*
        double time = -1;
        foreach(var child in mcr.FindChildren(method)){
            var childtime = child.TimeSpentInMethod.TotalSeconds;
            //var childtime = child.TimeSpentInMPIBlocking.TotalSeconds;
            time = childtime > time? childtime:time;
        }
        */
        return time;
    }
    static public double AverageTime(ISessionInfo SI,string method){
        var mcrs       = GetMCR(SI);
        double avtime = 0;

        foreach(var mcr in mcrs){
            avtime+=TimeOnProc(mcr,method);
        }
        avtime = avtime / mcrs.Count();
        return avtime;
    }
    static public double MaxTime(ISessionInfo SI,string method){
        var mcrs       = GetMCR(SI);
        double maxtime = -1;

        foreach(var mcr in mcrs){
            double time =TimeOnProc(mcr,method);    
            maxtime = time>maxtime?time:maxtime;
        }
        return maxtime;
    }
    static public object LSolverIter(ISessionInfo SI){
        var mcr          = GetMCR(SI)[0];
        string searchstr = "BoSSS.Solution.AdvancedSolvers.DirectSolver.Solve";
        var ndS = mcr.FindChildren(searchstr);
        var nd  = ndS.ElementAt(1);
        return nd.CallCount;
    }
    static public object NLSolverIter(ISessionInfo SI){
        var mcr          = GetMCR(SI)[0];
        string searchstr = "BoSSS.Solution.AdvancedSolvers.Newton.GlobalizedNewton";
        var ndS = mcr.FindChildren(searchstr);
        var nd  = ndS.ElementAt(0);
        return nd.CallCount;
    }
    static public object TotalRuntime(ISessionInfo SI){
        return MaxTime(SI,"root_frame");
    }
    static public object MatrixAssembly_time(ISessionInfo SI) {
        return MaxTime(SI,"BoSSS.Solution.XdgTimestepping.XdgBDFTimestepping.AssembleMatrixCallback");
    }
    static public object AggregationBaseInit_time(ISessionInfo SI) {
        return MaxTime(SI,"BoSSS.Solution.AdvancedSolvers.XdgAggregationBasis.Update");
    }
    static public object SolverInit_time(ISessionInfo SI) {
        return MaxTime(SI,"Slv Init");
    }
    static public object SolverRun_time(ISessionInfo SI) {
        return MaxTime(SI,"Slv Iter");
    }
    static public object DataIO_time(ISessionInfo SI){
        double sum = 0;
        sum+=MaxTime(SI,"BoSSS.Solution.Application`1.CreateOrLoadGrid");
        sum+=MaxTime(SI,"BoSSS.Solution.Application`1.SaveToDatabase");
        return sum;
    }
    static public object CGProjection_time(ISessionInfo SI){
        return MaxTime(SI,"BoSSS.Foundation.ConstrainedDGprojection.ConstrainedDGField_Global.ProjectDGField");
    }
    static public object SayeCompile_time(ISessionInfo SI){
        return MaxTime(SI,"Rule_Compilation_SayeGaussRuleFactory");
    }
    static public object StandardCompile_time(ISessionInfo SI){
        return MaxTime(SI,"Rule_Compilation_StandardQuadRuleFactory");
    }
    static public object AMR_time(ISessionInfo SI){
        return MaxTime(SI,"BoSSS.Solution.Application`1.DoMeshAdaption");
    }
    static public object LoadBal_time(ISessionInfo SI){
        return MaxTime(SI,"BoSSS.Solution.Application`1.DoLoadbalancing");
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.ComputeNodeNames.Count();
    }
    static public object NoOfNodes(ISessionInfo SI){
        return SI.ComputeNodeNames.GroupBy(s=>s).Count();
    }
    static public object MGDepth(ISessionInfo SI){
        var mcr = GetMCR(SI)[0];
        double NInit  = mcr.FindChildren("BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.Init").ToArray().Length;
        return NInit + 1;
    }
    static public object DOF(ISessionInfo SI){
        return SI.GetDOF("VelocityX")*3+SI.GetDOF("Pressure");
    }
    static public Func<ISessionInfo,object> FuncGenerator(string method){
        return delegate (ISessionInfo SI){
            return MaxTime(SI, method);
        };
    }
}

In [ ]:
wfmgr.Sessions.Select(s=>AddCols.LSolverIter(s))

In [ ]:
wfmgr.AdditionalSessionTableColums.Clear();
wfmgr.AdditionalSessionTableColums.Add("LSolverIter", AddCols.LSolverIter);
wfmgr.AdditionalSessionTableColums.Add("NLSolverIter", AddCols.NLSolverIter);
wfmgr.AdditionalSessionTableColums.Add("TotalRuntime", AddCols.TotalRuntime);
wfmgr.AdditionalSessionTableColums.Add("MatrixAssembly_time", AddCols.MatrixAssembly_time);
wfmgr.AdditionalSessionTableColums.Add("AggregationBaseInit_time", AddCols.AggregationBaseInit_time);
wfmgr.AdditionalSessionTableColums.Add("DataIO_time", AddCols.DataIO_time);
wfmgr.AdditionalSessionTableColums.Add("CGProjection_time", AddCols.CGProjection_time);
wfmgr.AdditionalSessionTableColums.Add("SayeCompile_time", AddCols.SayeCompile_time);
wfmgr.AdditionalSessionTableColums.Add("StandardCompile_time",AddCols.StandardCompile_time);
wfmgr.AdditionalSessionTableColums.Add("AMR_time",AddCols.AMR_time);
wfmgr.AdditionalSessionTableColums.Add("LoadBal_time",AddCols.LoadBal_time);
wfmgr.AdditionalSessionTableColums.Add("NoOfCores",AddCols.NoOfCores);
wfmgr.AdditionalSessionTableColums.Add("NoOfNodes",AddCols.NoOfNodes);
wfmgr.AdditionalSessionTableColums.Add("SolverInit_time", AddCols.SolverInit_time);
wfmgr.AdditionalSessionTableColums.Add("SolverRun_time", AddCols.SolverRun_time);
wfmgr.AdditionalSessionTableColums.Add("MGDepth",AddCols.MGDepth);
wfmgr.AdditionalSessionTableColums.Add("DOF",AddCols.DOF);

In [ ]:
string[] AdditionalTraces = new string[]{               
    "BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.AddSol",
    "BoSSS.Solution.AdvancedSolvers.Schwarz.Solve",
    "BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.MinimizeResidual",
    "BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.Residual",
    "BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.Solve",
    "overlap_scaling",
    "block_solve_level",
    "caching",
    "nocaching",
    "BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.VerivyCurrentResidual",
    "Restriction",
    "Prolongation",
    "BoSSS.Solution.AdvancedSolvers.DirectSolver.Solve"
};

In [ ]:
foreach(string Ftrace in AdditionalTraces){
    string[] Schnipsel = Ftrace.Split('.');
    int L = Schnipsel.Length;
    string ColumnName = "";
    if(L>1)
        ColumnName = String.Concat(Schnipsel[L-2],'.',Schnipsel[L-1]);
    else
        ColumnName = Schnipsel.Last();

    wfmgr.AdditionalSessionTableColums.Add(ColumnName,AddCols.FuncGenerator(Ftrace));
}

In [ ]:
var SessTab = wfmgr.SessionTable;
SessTab.GetColumnNames().OrderBy(s=>s)

In [ ]:
// The Session column can't be serialized,
// we have to remove it
List<string> AllCols = SessTab.GetColumnNames().ToList();
AllCols.Remove("Session");

In [ ]:
SessTab.GetColumnNames().Where(s=>s.Contains("Depth"))

In [ ]:
SessTab = SessTab.ExtractColumns(AllCols.ToArray());
// Filename
string dirpath = Directory.GetCurrentDirectory();
var now           = DateTime.Now;
SessTab.TableName = WFlowName;
string docpath    = Path.Combine(dirpath, SessTab.TableName + ".json");
SessTab.SaveToFile(docpath); docpath

In [ ]:
/*
var SubTab = SessTab.ExtractColumns("LSolverIter", "NLSolverIter", "TotalRuntime", "MatrixAssembly_time", "AggregationBaseInit_time", "DataIO_time", "CGProjection_time",
"SayeCompile_time", "StandardCompile_time","AMR_time","LoadBal_time","NoOfCores","NoOfNodes","SolverInit_time", "SolverRun_time", "MGDepth","DGdegree:Velocity*");
SubTab.ToCSVFile(dirpath+@"\table.csv");dirpath
*/